<a href="https://colab.research.google.com/github/rsadaphule/jhu-dnn/blob/main/6%F0%9F%8F%86%F0%9F%A6%A0Bacteria_Ravindra_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🦠Bacteria</font>](https://www.kaggle.com/c/3jul23jh-bacteria/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [11]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 3jul23jh-bacteria # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 3jul23jh-bacteria
100% 41.6M/41.6M [00:02<00:00, 15.6MB/s]
Using competition: 3jul23jh-bacteria
  teamId  teamName            submissionDate       score    
--------  ------------------  -------------------  -------  
10659307  Erik Wyatt-Nyquist  2023-07-09 15:29:19  0.98360  
10588337  🦠Baseline🐍          2023-06-21 13:27:43  0.93060  
10642678  Jeremy Shih         2023-07-04 21:48:48  0.93060  
10657369  Matt Sunday         2023-07-08 22:02:54  0.93060  
10659692  Kevin Key           2023-07-09 17:54:55  0.92140  


In [13]:
!pip -q install -U tensorflow_addons > log

In [14]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, tensorflow_addons as tfa
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 698 ms, sys: 1.76 ms, total: 700 ms
Wall time: 841 ms


In [15]:
df = pd.read_csv('XY_Bacteria.csv'); df

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,y
0,10,32,41,39,77,122,55,81,58,31,...,38,88,80,20,36,31,32,32,10,NaN
1,10,31,52,165,225,221,150,143,48,31,...,111,125,143,159,69,45,32,71,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,10,31,66,107,142,155,142,107,66,31,...,93,66,93,93,66,31,31,31,10,4.0
99999,10,32,41,21,45,89,45,60,67,31,...,97,82,74,80,52,31,32,7,10,2.0


In [16]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 287 entries, A0T0G0C10 to y
dtypes: float64(1), int64(286)
memory usage: 219.0 MB


In [17]:
vX = df.query('y!=y').drop('y', axis=1)    # slice a test sample
tXY = df.query('y==y')                     # slice training sample
tX, tY = tXY.drop('y', axis=1), tXY.y.astype(int)      # split into training I/O
vX  # test inputs
tX  # train inputs
print(tY.tolist()[:50]) # train outputs

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,10,32,41,39,77,122,55,81,58,31,...,27,38,88,80,20,36,31,32,32,10
1,10,31,52,165,225,221,150,143,48,31,...,76,111,125,143,159,69,45,32,71,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,10,31,66,107,142,156,143,107,66,31,...,420,296,303,421,302,306,31,31,31,10
9999,10,31,66,107,142,155,142,108,66,31,...,66,429,307,93,93,66,31,31,31,10


,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
10000,10,7,48,74,119,110,127,86,66,7,...,223,319,233,361,439,368,179,174,199,10
10001,10,31,66,107,142,156,142,108,66,31,...,66,539,302,93,424,434,31,313,31,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,10,31,66,107,142,155,142,107,66,31,...,66,93,66,93,93,66,31,31,31,10
99999,10,32,41,21,45,89,45,60,67,31,...,52,97,82,74,80,52,31,32,7,10


[3, 3, 4, 0, 4, 3, 4, 4, 3, 3, 2, 2, 4, 2, 2, 4, 4, 2, 2, 0, 2, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 4, 2, 3, 3, 2, 0, 1, 4, 4, 2, 3, 4, 3, 0, 3, 2, 2, 0]


In [18]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

I tried following appraches
1. Imputing missing values with Mean
2. Standard Scaling


| Model   |          Accuracy |
|---------|----------|
| Baseline Score   | 93%        |
| Imputing  and Scaling   | 98%     |



In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

tX_in = tXY.drop('y', axis=1)[:50000]
tY_in = tXY.y[:50000]

# Handle missing values
imputer = SimpleImputer(strategy='mean')  # or 'median' if the distribution is skewed
tX_imputed = imputer.fit_transform(tX_in)

# Scale features
scaler = StandardScaler()
tX_scaled = scaler.fit_transform(tX_in)

vX_scaled = scaler.fit_transform(vX)





In [31]:
from sklearn.decomposition import PCA

# Let's say we want to reduce the features to 100
pca = PCA(n_components=100)

# Fit and transform the data
tX_reduced = pca.fit_transform(tX_scaled)
vX_reduced = pca.fit_transform(vX_scaled)

**Student's answer:**

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

Below is a baseline model that produces the result on Kaggle leaderboard (LB).

In [23]:
'''
%%time
tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.RandomNormal(seed=0)

tX_in = tXY.drop('y', axis=1)[:50000]
tY_in = tXY.y[:50000]

m = keras.models.Sequential([
    Flatten(input_shape=[tX_in.shape[1]], name='input'),
    Dense(50, activation="relu", kernel_initializer=Init, name='hidden'),
    Dense(len(tY.unique()), activation='softmax', kernel_initializer=Init, name='output')])
m.summary()
m.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
hist = m.fit(tX_in, tY_in, epochs=10, validation_split=0.3, batch_size=32)
'''

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 286)               0         
                                                                 
 hidden (Dense)              (None, 50)                14350     
                                                                 
 output (Dense)              (None, 5)                 255       
                                                                 
Total params: 14,605
Trainable params: 14,605
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1094/1094 [==============================] - 4s 3ms/step - loss: 1.9177 - accuracy: 0.8244 - val_loss: 0.3711 - val_accuracy: 0.8694
Epoch 2/10
1094/1094 [==============================] - 4s 4ms/step - loss: 0.3310 - accuracy: 0.8864 - val_loss: 0.2848 - val_accuracy: 0.9045
Epoch 3/10
1094/1094 [====

In [32]:
%%time
tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.RandomNormal(seed=0)

#tX_in = tXY.drop('y', axis=1)[:50000]
#tY_in = tXY.y[:50000]

m = keras.models.Sequential([
    Flatten(input_shape=[tX_reduced.shape[1]], name='input'),
    Dense(50, activation="relu", kernel_initializer=Init, name='hidden'),
    Dense(len(tY.unique()), activation='softmax', kernel_initializer=Init, name='output')])
m.summary()
m.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
hist = m.fit(tX_reduced, tY_in, epochs=10, validation_split=0.3, batch_size=32)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 100)               0         
                                                                 
 hidden (Dense)              (None, 50)                5050      
                                                                 
 output (Dense)              (None, 5)                 255       
                                                                 
Total params: 5,305
Trainable params: 5,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1094/1094 [==============================] - 5s 4ms/step - loss: 0.3154 - accuracy: 0.9014 - val_loss: 0.1532 - val_accuracy: 0.9525
Epoch 2/10
1094/1094 [==============================] - 3s 2ms/step - loss: 0.1313 - accuracy: 0.9563 - val_loss: 0.1301 - val_accuracy: 0.9561
Epoch 3/10
1094/1094 [======

In [34]:
pY = pd.DataFrame(np.argmax(m.predict(vX_reduced), axis=-1), index=vX.index+1, columns=['y'])
ToCSV(pY.round(0).astype(int), '🦠Ravindra_v3')
print('Observed  y distribution: ', list((tXY.y.value_counts()/len(tXY)).round(3)))                # distribution of training target level
print('Predicted y distribution: ', list((pY.round(0).astype(int).value_counts()/len(pY)).round(3))) # distribution of test target level

313/313 [==============================] - 0s 1ms/step
Observed  y distribution:  [0.303, 0.302, 0.296, 0.05, 0.048]
Predicted y distribution:  [0.303, 0.302, 0.278, 0.077, 0.041]


# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).
1. ...
1. ...

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [21]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 46 sec


## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set

**Training observations**
1. Try clustering similar observations into fewer representative observations.
1. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain. Read [Analysis of Identification Method for Bacterial Species and Antibiotic Resistance Genes Using Optical Data From DNA Oligomers](https://www.frontiersin.org/articles/10.3389/fmicb.2020.00257/full) and cited/citing references.